# Praca domowa nr 1 - WUM2021L
### Autor: Bartosz Sawicki

In [ ]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns

from matplotlib import pyplot as plt
from scipy import stats
from pandas_profiling import ProfileReport

## Pobranie danych

In [ ]:
url = 'https://api.apispreadsheets.com/api/dataset/forest-fires/'
r = requests.get(url)
data = r.json()

df = pd.DataFrame.from_dict(data['data'], orient='columns')

## Ogólne informacje o zbiorze

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

 ### Co oznaczają skróty `FFMC`, `DMC`, `DC`, `ISI`?
 
 W skrócie: wskaźniki systemu __FWI__ (Fire Weather Index)
 
 - `FFMC` - The __Fine Fuel Moisture Code__ represents fuel moisture of forest litter fuels under the shade of a forest canopy. It is intended to represent moisture conditions for shaded litter fuels, the equivalent of 16-hour timelag. It ranges from 0-101. Subtracting the FFMC value from 100 can provide an estimate for the equivalent (approximately 10h) fuel moisture content, most accurate when FFMC values are roughly above 80.
 
- `DMC` - The __Duff Moisture Code__ represents fuel moisture of decomposed organic material underneath the litter. System designers suggest that it is represents moisture conditions for the equivalent of 15-day (or 360 hr) timelag fuels. It is unitless and open ended. It may provide insight to live fuel moisture stress.

- `DC` - The __Drought Code__, much like the Keetch-Byrum Drought Index, represents drying deep into the soil. It approximates moisture conditions for the equivalent of 53-day (1272 hour) timelag fuels. It is unitless, with a maximum value of 1000. Extreme drought conditions have produced DC values near 800.

- `ISI` - The __Initial Spread Index__ integrates fuel moisture for fine dead fuels and surface windspeed to estimate a spread potential. ISI is a key input for fire behavior predictions in the FBP system. It is unitless and open ended.

In [ ]:
df.columns

## Analiza zmiennej objaśnianej - `area`



In [ ]:
df['area'].describe()

In [ ]:
sns.histplot(df['area'], bins=round(1+3.322*np.log(df['area'].shape[0])), kde=True)
plt.show()

Większość obserwacji jest bliska 0. Rozkład prawostronny.

In [ ]:
print("Skośność: %f" % df['area'].skew())
print("Kurtoza: %f" % df['area'].kurt())

Potwierdza to obserwacje o prawostronnym i stromym rozkładzie.

### Transformacja `area`

Na stronie [zbioru danych](https://apispreadsheets.com/datasets/129) zasugerowano aby przetransformować zmienną logarytmem. Sprawdźmy. Zastosujemy x -> log(x+1) aby uniknąć problemów z nieskończonością.

In [ ]:
sns.histplot(df['area'].apply(lambda x: np.log(x+1)), bins=round(1+3.322*np.log(df['area'].shape[0])), kde=True)
plt.show()

Dodajmy tak przetransformowaną zmienną do zbioru danych. Sprawdzimy jak wygląda na wykresach zestawiona z innymi cechami.

In [ ]:
df['log_area'] = df['area'].apply(lambda x: np.log(x+1))
df.head()

Zobaczmy jak wyglądają dodatnie wartości `area`

In [ ]:
len(df[df['area']>0])

In [ ]:
sns.histplot(df.loc[df['area']>0, 'area'], bins=round(1+3.322*np.log(df[df['area']>0].shape[0])), kde=True)
plt.show()

Teraz możemy zastosować zwykły logarytm.

In [ ]:
sns.histplot(df.loc[df['area']>0, 'area'].apply(np.log), bins=round(1+3.322*np.log(df[df['area']>0].shape[0])), kde=True)
plt.show()

Rozkład bardziej przypomina rozkład normalny. Można rozważyć podzielenie zadania na 2 części. Najpierw klasyfikujemy czy pożar wybuchnie (`area` > 0), a później przybliżamy log(`area`) jakimś modelem.

## Utworzenie kolumn numerycznych kodujących dni tygodnia i miesiące

Dzięki temu będziemy mogli wyliczyć miary statystyczne i zobczyć histogramy.

- dni tygodnia: 1=poniedziałek, ... , 7=niedziela
- miesiące: 1=styczeń, ... , 12=grudzień

In [ ]:
df['month_num'] = pd.to_datetime(df.month, format='%b').dt.month
weekdays = {'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5, 'sat':6, 'sun':7}
df['day_num'] = df['day'].map(weekdays)
df.head()

## Histogramy zmiennych

In [ ]:
df.hist(bins = 40, figsize = (18,12))
plt.show()

- `rain` bardzo dużo obserwacji ma wartość bliską 0. Może warto usunąć tę kolumnę.

## Korelacje cech

In [ ]:
corr = df.drop(['month_num', 'day_num'], axis = 1).corr()
_, __ = plt.subplots(figsize=(12,9))
sns.heatmap(corr, vmin=-1,annot=True)
plt.show()

- `temp` odwrotnie skorelowany z `RH`. Im cieplej tym względna wilgotność niższa.

In [ ]:
sns.scatterplot(x = df['temp'], y = df['RH'])
plt.show()

- wzajemnie skorelowane `ISI`, `temp`, `FFMC`, `DMC`, `DC`.

In [ ]:
cols = ['ISI', 'temp', 'FFMC', 'DMC', 'DC']
sns.pairplot(df[cols])
plt.show()

## Współrzędne geograficzne



Mapa przedstawiająca podział parku na sektory

In [ ]:
import matplotlib.image as mpimg
image = mpimg.imread("images/download.jpeg")
plt.imshow(image)
plt.show()

In [ ]:
def create_heatmap(agg_fun):
    geo_df = df.loc[:,['X','Y','area']].groupby(['X', 'Y'], as_index = False).agg(agg_fun)
    geo_df_pivot = geo_df.pivot(index = 'X', columns = 'Y', values = 'area')
    geo_df_pivot[geo_df_pivot.isna()] = 0
    sns.heatmap(geo_df_pivot, cmap = 'YlOrRd').set_title(agg_fun + ' of areas by coordinates')
    plt.show()

create_heatmap('sum')
create_heatmap('count')
create_heatmap('mean')
create_heatmap('std')

Widać, że największe i najczęstsze pożary są w prostokątnym pasie leżącym wzdłuż przekątnej terenu. Stąd też współczynnik korelacji `X` i `Y` to 0.54. Największa średnia powierzchnia pożaru jest w prostokącie (8,8), gdzie odnotowano mniej niż 10 pożarów. Odchylenie standardowe spalonej powierzchni w tym rejonie też jest niewielkie. Sugeruje to, że było tam kilka dużych pożarów.

## Dane w czasie

`DC` opisuje poziom suszy. Logiczne jest, że wraz z nadejściem lata ten wskażnik rośnie<br/>

In [ ]:
import matplotlib.image as mpimg
image = mpimg.imread("images/437-cffdrs-fuel-moisture-codes-graph.png")
plt.imshow(image)
plt.show()

In [ ]:
sns.scatterplot(x = df['month_num'], y = df['DC'])
plt.show()

In [ ]:
month_df = df.loc[:,['month_num', 'month', 'area']].groupby(['month_num', 'month']).agg([np.sum, np.size, np.mean]).reset_index()
month_df.loc[:,'area']

f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 20))

sns.barplot(x = month_df['month'], y = month_df.loc[:,'area']['sum'], ax = ax1)
ax1.title.set_text('Sum of areas by month')
ax2.set_ylabel('sum of areas')

sns.barplot(x = month_df['month'], y = month_df.loc[:,'area']['size'], ax = ax2)
ax2.title.set_text('Number of fires by month')
ax2.set_ylabel('number')

sns.barplot(x = month_df['month'], y = month_df.loc[:,'area']['mean'], ax = ax3)
ax3.title.set_text('Mean of fire areas by month')
ax3.set_ylabel('mean of areas')

plt.show()

Nie jest zaskoczeniem, że późnym latem pożarów jest najwięcej i suma spalonych obszarów jest największa. Warto zauważyć, że średnia powierzchnia pożaru nie zależy aż tak bardzo od pory roku.

In [ ]:
weekday_df = df.loc[:,['day_num', 'day', 'area']].groupby(['day_num', 'day']).agg([np.sum, np.size, np.mean]).reset_index()
weekday_df.loc[:,'area']

f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 20))

sns.barplot(x = weekday_df['day'], y = weekday_df.loc[:,'area']['sum'], ax = ax1)
ax1.title.set_text('Sum of areas by day')
ax2.set_ylabel('sum of areas')

sns.barplot(x = weekday_df['day'], y = weekday_df.loc[:,'area']['size'], ax = ax2)
ax2.title.set_text('Number of fires by day')
ax2.set_ylabel('number')


sns.barplot(x = weekday_df['day'], y = weekday_df.loc[:,'area']['mean'], ax = ax3)
ax3.title.set_text('Mean of fire areas by day')
ax3.set_ylabel('mean')

plt.show()

Najwięcej pożarów wybucha w okolicach weekendów. Może to wynikać ze wzmożonej obecności turystów w dni wolne.

## Narzędzie do automatycznej eksploracji - pandas profiler

In [ ]:
profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)

In [ ]:
profile.to_notebook_iframe()

### Wnioski z raportu

- są 4 zdublowane obserwacje. Prawdopodobnie jest to wynik błędu i lepiej je usunąć.

### Opinia o narzędziu

- automatyzuje wstępny etap eksploracji
- wyłapuje anomalia w danych, takie jak dużo zerowych wartości, duplikaty
- po przeczytaniu raportu można zyskać intuicję co do dalszej analizy
- profiler nie zrobi za nas bardziej zaawansowanych wykresów i podsumowań